# 03. Ch 03. Data Preparation - 03. 실습 데이터 살펴보기

In [ ]:
셔플링 후 트레인 밸리드 테스트 셋으로 나눠주기
이후 토크나이제이션

-텍스트 파일 tsv 파일로 합치기
cat ./raw/*.txt > corpus.tsv
-코퍼스 문장 수 확인
wc -l ./corpus.tsv
-셔플링
shuf ./corpus.tsv > ./corpus.shuf.tsv 
-트레인, 밸리드, 테스트 셋 스플릿
head -n 450000 ./corpus.shuf.tsv > corpus.shuf.train.tsv ; tail -n 205340 ./corpus.shuf.tsv | head -n 100000 > ./corpus.shuf.valid.tsv 

tail -n 105340 ./corpus.shuf.tsv > ./corpus.shuf.test.tsv

-토크나이제이션 해주기 위해 한국어 따로 영어 따로 나누기
cut -f1 ./corpus.shuf.train.tsv > corpus.shuf.train.ko ; cut -f2 ./corpus.shuf.train.tsv > corpus.shuf.train.en

head -n 3 ./corpus.shuf.train.*

cut -f1 ./corpus.shuf.test.tsv > ./corpus.shuf.test.ko ; cut -f2 ./corpus.shuf.test.tsv > ./corpus.shuf.test.en

cut -f1 ./corpus.shuf.valid.tsv > ./corpus.shuf.valid.ko ; cut -f2 ./corpus.shuf.valid.tsv > ./corpus.shuf.valid.en

head -n 3 ./corpus.shuf.*.ko

head -n 3 ./corpus.shuf.*.en

# 04. Ch 03. Data Preparation - 04. Review Preprocessing

In [ ]:
<NLP 프로젝트 워크플로>
문제 정의 -> 데이터 수집 -> 데이터 전처리 및 분석 -> 알고리즘 적용 -> 평가 -> 배포

<전처리 워크플로>
데이터 수집 -> 정제 -> 레이블링 -> 토크나이제이션 -> 서브워드 세그멘테이션 -> Batchify

토크나이제이션만 하거나 서브워드 세그멘테이션만 하거나 둘 다 하거나
프리트레인 랭귀지 모델 같은 경우에는 서브워드 세그멘테이션만 하는 경우도 많다.
둘 다 하는 걸 추천
애초 영어 같은 경우에는 띄어쓰기가 잘 돼 있다.
한국어 교착어고 띄어쓰기가 개판이라는 언어적 특징이 있기 떄문에 둘 다 하는 것을 선호한다는 것.
한국어에 띄어쓰기 도입된 것이 근대. 여전히 제대로 정착 안 됨.
성능의 미미한 차이가 있을 뿐. 했을 때 안 되던게 되거나 안 했을 때 되던 게 안 되는 일은 벌어지지 않음.

서브워드 세그멘테이션 수행하면 OoV를 제거할 수 있다는 장점이 있음.

<Cleaning>
-기계적인 노이즈 제거
전각문자 변환(전각문자를 반각문자로)
태스크에 따른 (전형적인) 노이즈 제거(특수문자, 문장부호, 이모티콘 등)
-Interactive 노이즈 제거
코퍼스의 특성에 따른 노이즈 제거(find-replace 활용)
작업자가 상황을 확인하며 작업 수행
-Therefore,
전처리 과정은 태스크와 언어, 도메인과 코퍼스의 특성에 따라 다르다
시간과 품질 사이의 트레이드 오프
따라서 전처리 중에서도 특히 데이터 노이즈 제거의 경우 많은 노하우가 필요

<토크나이제이션>
-한국어의 경우
1)접사를 분리해 희소성을 낮추고
2)띄어쓰기를 통일하기 위해 토크나이제션을 수행
-굉장히 많은 POS Tagger가 존재하는데
전형적인 쉬운 문장(표준 문법을 따르며, 구조가 명확한 문장)의 경우 성능이 비슷함
하지만 신조어나 고유명사를 처리하는 능력이 다름
따라서 주어진 문제에 맞는 정책을 가진 tagger를 선택해 사용해야 함

mecab 선호

<서브워드 세그멘테이션>
-BPE 압축 알고리즘을 통해 통계적으로 더 작은 의미 단위(서브워드)로 분절 수행
-BPE를 통해 OoV를 없앨 수 있으며, 이는 성능상 매우 큰 이점으로 작용
텍스트 분류는 BPE 굳이 필요 없을 수 있다
하지만 NLG에서 OoV가 나타나는 순간 큰 일이 나기 때문에 이를 제거할 수 있다는 것만해도 큰 이점으로 작용
-한국어의 경우
띄어쓰기가 제멋대로인 경우가 많으므로 normalization 없이 바로 서브워드 세그멘테이션을 적용하는 것은 위험
따라서 형태소 분석기를 통한 tokenization을 진행한 이후 서브워드 세그멘테이션을 적용하는 것을 권장

<Batchify>
토치텍스트 활용해볼 것이다

# 05. Ch 03. Data Preparation - 05. 실습 Tokenization

In [ ]:
mecab(토크나이제이션) 후 BPE(서브워드 세그멘테이션) 할 것.
데이터는 이미 정제까지 돼 있는 상태.

ll = ls -al 
head -n 5 ./data/corpus.shuf.train.ko | mecab -O wakati
cat ./data/corpus.shuf.test.ko | mecab -O wakati > ./data/corpus.shuf.test.tok.ko

cat ./data/corpus.shuf.test.ko | mecab -O wakati -b 99999 | python ./post_tokenize.py ./data/corpus.shuf.test.ko
> ./data/corpus.shuf.test.tok.ko
#b: 버퍼 사이즈, 버퍼를 넣어주지 않으면 에러가 날 수도 있다.
# post_tokenize.py 통해 언더스코어 붙이기
# 각 단계마다 언더스코어를 넣어줘서 디토크나이제이션을 도와준다.
# 여기서 언더스코어는 일반 언더스코어가 아님. 특수문자
# 언더스코어 있던 부분이 원래 띄어쓰기 있던 곳. 그냥 띄어쓰기 돼 있는 곳은 띄어쓰기가 없던 곳.

-디토크나이즈
head -n 5 ./data/corpus.shuf.test.tok.ko | python detokenizer.py

-나머지 파일 토크나이제이션
cat ./data/corpus.shuf.train.ko | mecab -O wakati | python post_tokenize.py ./data/corpus.shuf.train.ko > ./data/corpus.shuf.train.tok.ko

cat ./data/corpus.shuf.train.en | python tokenizer.py | python ./post_tokenize.py ./data/corpus.shuf.train.en > ./data/corpus.shuf.train.tok.en &

cat ./data/corpus.shuf.valid.ko | mecab -O wakati | python post_tokenize.py ./data/corpus.shuf.valid.ko > ./data/corpus.shuf.valid.tok.ko

cat ./data/corpus.shuf.valid.en | python tokenizer.py | python ./post_tokenize.py ./data/corpus.shuf.valid.en > ./data/corpus.shuf.valid.tok.en &

aihub에서 엑셀 파일 받고 -> 텍스트 파일로 옮기고 -> 탭 명령어를 통해 corpus.tsv 하나로 합치고 -> 셔플링
-> 120만/20만/20만 테스트 밸리드 트레인셋 나누고 -> cut을 통해 ko en 이라는 확장자 파일 만든 다음 -> 토크나이제이션 수행

명령어 뒤에 & 붙이면 병렬로 실행할 수 있다.
jobs 명령어 치면 현재 돌아가고 있는 작업 볼 수 있음.

-라인 수로 최종 확인
wc -l ./data/corpus.shuf.*.tok.*

여기까지만해도 잘 동작한다.

# 06. Ch 03. Data Preparation - 06. 실습 Subword Segmentaion

In [ ]:
cat README.md
https://github.com/kh-kim/subword-nmt
원래 서브워드 nmt를 조금 가공한 것.
우리는 토크나이제이션 이후 서브워드 세그멘테이션하는 것이기 때문에 그에 맞게 변형했다.

git clone https://github.com/kh-kim/subword-nmt

서브워드 세그멘테이션 하게 되면 원래 언더스코어 있던 곳에는 언더스코어가 한 번 더 들어가게 된다.

정제가 잘 된 코퍼스는 BPE만 먹여도 잘 될 것이다
그런데 일반적으로 크롤링을 해서 얻은 코퍼스는 띄어쓰기가 굉장히 잘 안 돼 있음
이런 코퍼스에 바로 BPE를 먹이면 띄어쓰기에 대해서 통계적으로 BPE가 들어가게 되는건데
노이즈가 껴서 통계가 고르지 않게될 것이다.
뜻하지 않은 대로 세그멘테이션 할 수 있기 때문에 토크나이제이션(mecab) 하고 나서 다음에 서브워드 세그멘테이션 하는 것을 추천.
영어는 쉼표나 마침표는 같은 것만 띄어주고… 영어는 워낙 띄어쓰기 잘 돼 있어서.

learn_bpe.py와 apply_bpe.py를 주로 사용할 것

-learn_bpe 옵션 확인
python ./subword-nmt/learn_bpe.py --asdf

python ./subword-nmt/learn_bpe.py --input ./data/corpus.shuf.train.tok.en --output bpe.en.model --symbols 50000 --v
# symbols는 몇번의 머지 오퍼레이션이 일어날 거냐
# verbose를 켜서 진행 정도를 보면서

head bpe.en.model
tail bep.en.model
# 뭘 합쳤는지 차례대로 써져 있는 것. merge instruction이 써져 있는 것.

wc -l ./bpe.en.model

-트레인 데이터에 대해 학습
python ./subword-nmt/learn_bpe.py --input ./data/corpus.shuf.train.tok.ko --output bpe.ko.model --symbols 30000 --verbose
# 한국어는 영어에 비해서 단어가 복잡하기 때문에 3만 문장 정도면 충분함
# 이렇게 merge하는 횟수도 하이퍼파라미터
# 나중에 우리가 원하는 보카뷰러리의 사이즈가 나오는지 확인해볼 수 있음.
# 예를 들어 관계자도 관계와 자를 띄어써야 하는 것 아니냐고 하면 심볼 숫자를 줄이면 됨.

트레인 파일에 대해 학습해 트레인 셋에 그대로 적용하고
밸리드와 테스트 셋에도 트레인셋에서 배운 것을 그대로 적용한다.
이게 정석
전체 파일에 대해 학습해 통계를 내면 테스트 셋에 대한 통계까지 모두 공개가 되는 것.
제대로 하려면 트레인 셋에 대해서만 하는 게 맞다.

-나머지 파일에 대해 apply_bpe 실행
cat ./data/corpus.shuf.train.tok.ko | python subword-nmt/apply_bpe.py -c ./bpe.ko.model > ./data/corpus.shuf.train.tok.bpe.ko ; cat ./data/corpus.shuf.valid.tok.ko | python subword-nmt/apply_bpe.py -c ./bpe.ko.model > ./data/corpus.shuf.valid.tok.bpe.ko ; cat ./data/corpus.shuf.test.tok.ko | python subword-nmt/apply_bpe.py -c bpe.ko.model > ./data/corpus.shuf.test.tok.bpe.ko

# 너무 오래 걸리므로

jobs

# 백그라운드 실행
bg 1
bg 2
bg 3

-en 파일에 대해 실행
cat ./data/corpus.shuf.train.tok.en | python subword-nmt/apply_bpe.py -c ./bpe.en.model > ./data/corpus.shuf.train.tok.bpe.en ; cat ./data/corpus.shuf.valid.tok.en | python subword-nmt/apply_bpe.py -c ./bpe.en.model > ./data/corpus.shuf.valid.tok.bpe.en ; cat ./data/corpus.shuf.test.tok.en | python subword-nmt/apply_bpe.py -c ./bpe.en.model > ./data/corpus.shuf.test.tok.bpe.en

-확인
head -n 3 ./data/corpus.shuf.*.tok.bpe.ko
head -n 3 ./data/corpus.shuf.*.tok.bpe.en

-최종 확인
wc -l ./data/corpus.shuf.*.tok.bpe.*

history | grep apply_bpe